# Creating images

To run a YOLO model with transfer learning (just for classification, without bounding boxes for now), we're going to need to create a dataset for classification of the following format:
- Folder directory (containing training data)
    - Class 1 (tornado)
        - Many images of tornadoes (each file one image)
    - Class 2 (not tornado)
        - Many images of not tornadoes (each file one image)
     
The plan is to run a script that will take a year of data and save the images.
Thankfully, the TorNet code does have a `plot_radar` function that we can use to create a plot with the radar imagery
from each channel, which we can then use to run the YOLO classification model.
We'll save the data into the two class folders, image by image, as needed.

In [2]:
import os
import subprocess
import sys
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tornet.data.loader import TornadoDataLoader
from tornet.display.display import plot_radar
from tornet.data.constants import ALL_VARIABLES

plt.ioff()

TORNET_DATA_INPUT_FOLDER = "/mnt/c/users/handypark/Documents/Grad_School_Courses/CS_230/tornet"
catalog_path = os.path.join(TORNET_DATA_INPUT_FOLDER,'catalog.csv')
data_type='train'
years = [2015]
year = years[0]

# Grab only data for the given year and train/test
catalog = pd.read_csv(catalog_path, parse_dates=['start_time','end_time'])
catalog = catalog[catalog['type'] == data_type]
catalog = catalog[catalog.start_time.dt.year.isin(years)]
catalog = catalog.sample(frac=1, random_state=5678)

# tornadoes have an EF number labeled while non-tornadoes have -1
catalog_not_tornadoes = catalog[catalog.ef_number == -1]
catalog_tornadoes = catalog[catalog.ef_number != -1]

tornado_list = [os.path.join(TORNET_DATA_INPUT_FOLDER, f) for f in catalog_tornadoes.filename]
not_tornado_list = [os.path.join(TORNET_DATA_INPUT_FOLDER, f) for f in catalog_not_tornadoes.filename]

vars_to_plot=['DBZ', 'VEL', 'KDP', 'RHOHV', 'ZDR', 'WIDTH']
n_frames=1
tilt_last=True
D_ROOT = "/mnt/d/tornetExamples"

tornado_data = TornadoDataLoader(tornado_list, variables=vars_to_plot, n_frames=n_frames)
not_tornado_data = TornadoDataLoader(not_tornado_list, variables=vars_to_plot, n_frames=n_frames)

for didx in range(len(tornado_data)):
    x = tornado_data[didx]

    # lower tilt angle (all 6 radar images)
    fig = plt.figure(figsize=(12,6),edgecolor='k')
    plot_radar(x,
               fig=fig,
               channels=vars_to_plot,
    	       sweep_idx=[0, 0, 0, 0, 0, 0],
               include_cbar=True,
               time_idx=-1, 
               n_rows=2, n_cols=3)
    plt.savefig('{}/tmp/fig_low_{}.png'.format(D_ROOT, didx))
    plt.close()

    # upper tilt angle (all 6 radar images)
    fig_2 = plt.figure(figsize=(12,6),edgecolor='k')
    plot_radar(x,
               fig=fig_2,
               channels=vars_to_plot,
    	       sweep_idx=[1, 1, 1, 1, 1, 1],
               include_cbar=True,
               time_idx=-1, 
               n_rows=2, n_cols=3)
    plt.savefig('{}/tmp/fig_high_{}.png'.format(D_ROOT, didx))
    plt.close()

    # take the two tilt angles and concatenate the two sets of images together into one image
    low_sweep = cv2.imread('{}/tmp/fig_low_{}.png'.format(D_ROOT, didx))
    high_sweep = cv2.imread('{}/tmp/fig_high_{}.png'.format(D_ROOT, didx))
    concatenated = np.concatenate((low_sweep, high_sweep), axis=0)
    cv2.imwrite('{}/train/{}/tornado/{}_fig_{}.png'.format(D_ROOT, year, year, didx), concatenated)    

for didx in range(len(not_tornado_data)):
    # lower tilt angle (all 6 radar images)
    x = not_tornado_data[didx]
    fig = plt.figure(figsize=(12,6),edgecolor='k')
    plot_radar(x,
               fig=fig,
               channels=vars_to_plot,
    	       sweep_idx=[0, 0, 0, 0, 0, 0],
               include_cbar=True,
               time_idx=-1, 
               n_rows=2, n_cols=3)
    plt.savefig('{}/tmp/fig_low_{}.png'.format(D_ROOT, didx))
    plt.close()

    # upper tilt angle (all 6 radar images)
    fig_2 = plt.figure(figsize=(12,6),edgecolor='k')
    plot_radar(x,
               fig=fig_2,
               channels=vars_to_plot,
    	       sweep_idx=[1, 1, 1, 1, 1, 1],
               include_cbar=True,
               time_idx=-1, 
               n_rows=2, n_cols=3)
    plt.savefig('{}/tmp/fig_high_{}.png'.format(D_ROOT, didx))
    plt.close()

    # take the two tilt angles and concatenate the two sets of images together into one image
    low_sweep = cv2.imread('{}/tmp/fig_low_{}.png'.format(D_ROOT, didx))
    high_sweep = cv2.imread('{}/tmp/fig_high_{}.png'.format(D_ROOT, didx))
    concatenated = np.concatenate((low_sweep, high_sweep), axis=0)
    cv2.imwrite('{}/train/{}/not_tornado/{}_fig_{}.png'.format(D_ROOT, year, year, didx), concatenated)      